In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import os
from os.path import join

In [33]:
basedir = './datasets/andromeda-forums'

scrapes = os.listdir(basedir)[1:] #get list of scrape directories

In [54]:
#collect relative urls for each page scraped
thread_urls = []

for scrape in scrapes:
    for url in os.listdir(join(basedir, scrape)):
        if 'topic' in url:
            path = join(join(basedir, scrape), url)
            thread_urls.append(path)

In [93]:
#extract posts and poster names
posts_list = []
thread_ids = iter(range(10000))
for thread in thread_urls:
    with open(thread, 'r') as content:
        soup = BeautifulSoup(content, 'lxml')
        t_id = thread_ids.next()
        
        poster_tags = soup.findAll('div', class_='poster')
        
        posters = []
        for tag in poster_tags: #get all poster names
            posters.append(list(tag.children)[1].text.replace('\n', '').encode('utf-8'))
        
        post_tags = soup.findAll('div', class_='post')
        content = []
        for tag in post_tags: #get all poster names
            content.append(tag.text.replace('\n', '').encode('utf-8'))
        
        posts = zip(posters, content, [t_id for i in range(len(posters))])
        posts_list.extend(posts)
        
posts_list = np.array(posts_list)

In [100]:
posts_df = pd.DataFrame(posts_list, columns=['user', 'content', 'thread'])
posts_df.head()

,user,content,thread
0,JulesWinnfield,Just for safety issues.,1
1,LostTemple,"Yes, this is something that is being worked on...",1
2,NOWTHATSTHESTUFF,just check out my listings dont go through the...,2
3,drgreentreez,glad to reunite with such a standout vendor of...,2
4,drgreentreez,I will be a loyal and active customer of any...,3


In [101]:
#dump to csv
posts_df.to_csv('./datasets/andromeda_posts.csv')